In [1]:
from sympy import *

In [2]:
var('A1 A2 A3 A4')
A_cumsum = Matrix([[0, 0, 0, 0,0], 
                   [A1, 0, 0,0,0],
                   [A2, A2-A1,0,0,0],
                   [A3,A3-A1,A3-A2,0,0],
                   [A4, A4-A1, A4-A2, A4-A3,0],
                  ]).applyfunc(lambda x: exp(x) if x else 0)
A_cumsum1 = Matrix([[0,0,0],
                   [A1,0,0],
                   [A2, A2-A1, 0]]).applyfunc(lambda x: exp(x) if x else 0)
A_cumsum2 = Matrix([[0,0,0],
                   [A3,0,0],
                   [A4,A4-A3,0]]).applyfunc(lambda x: exp(x) if x else 0)

In [3]:
pprint(A_cumsum)
pprint(A_cumsum1)
pprint(A_cumsum2)

⎡ 0       0          0          0      0⎤
⎢                                       ⎥
⎢ A₁                                    ⎥
⎢ℯ        0          0          0      0⎥
⎢                                       ⎥
⎢ A₂   -A₁ + A₂                         ⎥
⎢ℯ    ℯ              0          0      0⎥
⎢                                       ⎥
⎢ A₃   -A₁ + A₃   -A₂ + A₃              ⎥
⎢ℯ    ℯ          ℯ              0      0⎥
⎢                                       ⎥
⎢ A₄   -A₁ + A₄   -A₂ + A₄   -A₃ + A₄   ⎥
⎣ℯ    ℯ          ℯ          ℯ          0⎦
⎡ 0       0      0⎤
⎢                 ⎥
⎢ A₁              ⎥
⎢ℯ        0      0⎥
⎢                 ⎥
⎢ A₂   -A₁ + A₂   ⎥
⎣ℯ    ℯ          0⎦
⎡ 0       0      0⎤
⎢                 ⎥
⎢ A₃              ⎥
⎢ℯ        0      0⎥
⎢                 ⎥
⎢ A₄   -A₃ + A₄   ⎥
⎣ℯ    ℯ          0⎦


In [6]:
var('hi, hw, hx, hy, hz')
states = Matrix([0, hw, hx, hy, hz])
states1 = Matrix([0,hw,hx])
states2 = Matrix([0,hy,hz])
hi = 0

In [7]:
pprint(A_cumsum.multiply(states))
pprint(A_cumsum1.multiply(states1))
pprint(A_cumsum2.multiply(states2))

⎡                    0                     ⎤
⎢                                          ⎥
⎢                    0                     ⎥
⎢                                          ⎥
⎢                   -A₁ + A₂               ⎥
⎢               hw⋅ℯ                       ⎥
⎢                                          ⎥
⎢           -A₁ + A₃       -A₂ + A₃        ⎥
⎢       hw⋅ℯ         + hx⋅ℯ                ⎥
⎢                                          ⎥
⎢    -A₁ + A₄       -A₂ + A₄       -A₃ + A₄⎥
⎣hw⋅ℯ         + hx⋅ℯ         + hy⋅ℯ        ⎦
⎡     0      ⎤
⎢            ⎥
⎢     0      ⎥
⎢            ⎥
⎢    -A₁ + A₂⎥
⎣hw⋅ℯ        ⎦
⎡     0      ⎤
⎢            ⎥
⎢     0      ⎥
⎢            ⎥
⎢    -A₃ + A₄⎥
⎣hy⋅ℯ        ⎦
